In [239]:
import pandas as pd
import numpy as np
from sodapy import Socrata

## DON'T RUN BELOW CELL

In [253]:
#generate Synthatic Data
df2 = pd.DataFrame(np.random.rand(100, 3)*5,
                   columns=['width', 'length', 'depth'])
#save generic data to csv file
df2.to_csv('Synthetic_Data.csv',sep=',',header=True,index=False)

In [240]:
#load data from csv file
pd.options.mode.chained_assignment = None  # default='warn' turn off warnings
data = pd.DataFrame()
def load_input_data():
    global data
    data = pd.read_csv('Input_data_Synthetic.csv',index_col=False)
    data.drop_duplicates(inplace=True)

In [254]:
def get_traffic_density():
   

    global data
    
    # Unauthenticated client only works with public data sets. Note 'None'
    # in place of application token, and no username or password:
    client = Socrata("data.cityofchicago.org", None)

    # Example authenticated client (needed for non-public datasets):
    # client = Socrata(data.cityofchicago.org,
    #                  MyAppToken,
    #                  userame="user@example.com",
    #                  password="AFakePassword")

    # First 2000 results, returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    results = client.get("u77m-8jgp", limit=10000)

    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)
    results_df = results_df.convert_objects(convert_numeric=True)

    data = pd.merge(data, results_df, how='left', on=['latitude', 'longitude'])

In [255]:
#this is mock algorithmm we probably gotta redefine it.
def the_almighty_algorithm(row):
    res = 5*row['width']+0.3*row['length']+ 20*row['depth']+np.log(row['total_passing_vehicle_volume'])
    return res

In [262]:
#apply algorithm on data
def score():
    global data
    data['score'] = data.apply(lambda row: the_almighty_algorithm(row), axis=1)
    data['binned_score']= 'Urgent'
    data['binned_score'][data['score']<data['score'].quantile(q=0.80)]='Required'
    data['binned_score'][data.score<data.score.quantile(q=0.60)]='Priority'
    data['binned_score'][data.score<data.score.quantile(q=0.350)]='Recommended'
    data = data[['width', 'length', 'depth', 'latitude','longitude', 'score','binned_score', 'date_of_count', 'id', 'location', 'street','total_passing_vehicle_volume','traffic_volume_count_location_address','vehicle_volume_by_each_direction_of_traffic']]

In [263]:
def __main():
    global data
    print("loading data....")
    load_input_data()
    data.head()
    print("Density Traffic info is being generated...")
    get_traffic_density()
    data.head()
    print("scoring....")
    score()
    #data = data.sort_values(by ='score',axis=0,ascending=False)
    data.head()

In [264]:
__main()

loading data....
Density Traffic info is being generated...
scoring....


C:\Users\Amer\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [265]:
data.head(n=100)

,width,length,depth,latitude,longitude,score,binned_score,date_of_count,id,location,street,total_passing_vehicle_volume,traffic_volume_count_location_address,vehicle_volume_by_each_direction_of_traffic
0,4.804688,1.897844,4.345184,41.908868,-87.804756,121.987757,Urgent,2006-12-20T02:00:00.000,728,"{'type': 'Point', 'coordinates': [-87.804756, ...",North Ave,36000,7168 West,East Bound: 17100 / West Bound: 18900
1,2.565842,4.414431,4.838154,41.684228,-87.702778,120.565225,Urgent,2006-08-22T02:00:00.000,816,"{'type': 'Point', 'coordinates': [-87.702778, ...",115th St,15500,3254 West,East Bound: 8100 / West Bound: 7400
2,2.427792,4.751201,4.902301,41.764877,-87.666635,121.199121,Urgent,2006-03-09T02:00:00.000,2,"{'type': 'Point', 'coordinates': [-87.666635, ...",71st Street,14600,1708 West,East Bound: 6900 / West Bound: 7700
3,4.473406,3.259579,4.113133,41.765008,-87.657067,115.318681,Urgent,2006-02-28T02:00:00.000,3,"{'type': 'Point', 'coordinates': [-87.657067, ...",71st Street,16500,1275 West,East Bound: 7800 / West Bound: 8700
4,4.788331,0.139403,4.666990,41.765153,-87.647751,127.132450,Urgent,2006-02-28T02:00:00.000,5,"{'type': 'Point', 'coordinates': [-87.647751, ...",71st Street,18200,920 West,East Bound: 8800 / West Bound: 9400
5,4.587650,1.123624,4.466826,41.765204,-87.644371,122.592310,Urgent,2006-02-28T02:00:00.000,6,"{'type': 'Point', 'coordinates': [-87.644371, ...",71st Street,21600,758 West,East Bound: 10400 / West Bound: 11200
6,3.755596,2.403649,4.364371,41.765644,-87.618476,116.601157,Urgent,2006-02-28T02:00:00.000,7,"{'type': 'Point', 'coordinates': [-87.618476, ...",71st St,18300,240 East,East Bound: 9000 / West Bound: 9300
7,4.000667,3.593559,3.932413,41.766260,-87.574226,108.789182,Urgent,2006-03-09T02:00:00.000,9,"{'type': 'Point', 'coordinates': [-87.574226, ...",71st St,8600,2050 East,East Bound: 3600 / West Bound: 5000
8,3.146943,3.914738,4.276384,41.739836,-87.551476,111.647162,Urgent,2006-03-07T02:00:00.000,10,"{'type': 'Point', 'coordinates': [-87.551476, ...",Commercial Ave,10000,8539 South,North Bound: 5000 / South Bound: 5000
9,4.147154,2.305761,3.970687,41.732725,-87.551320,110.100377,Urgent,2006-03-07T02:00:00.000,11,"{'type': 'Point', 'coordinates': [-87.55132, 4...",Commercial Ave,10500,8933 South,North Bound: 4900 / South Bound: 5600
